# Entwicklungsaktivität in Bezug auf die beteiligten Entwickler:innen

Diese Analyse beschäftigt sich mit der Darstellung der Entwicklungsaktivitäten in Bezug auf die beteiligten Entwickler:innen.
Sie werden pro Tag aggregiert dargestellt. Die verwendete Metrik sind die getätigten Commits der Entwickler:innen über alle Projekte hinweg.

## Laden relevanter Daten aus der Datenbank

Als erster Schritt werden die für die Analyse relevanten Daten aus der Datenbank geladen. Diese werden bereits den Entwickler:innen zugewiesen.

In [1]:
import shared.paths as paths
import shared.locals as locals
from datetime import datetime, timedelta
from tinydb import where
from helpers.config import Configuration
from database.database import Database
from database.models import Commit, User

database = Database(paths.DATABASE_PATH, cached=True)
config = Configuration.from_input_file(paths.CONFIG_PATH)

start_date = datetime.utcfromtimestamp(config.start_timestamp)
limit_date = datetime.utcfromtimestamp(config.limit_timestamp)

delta = limit_date - start_date
days = [(start_date+timedelta(days=i)).date() for i in range(delta.days + 1)]

users = database.all(User.LABEL)
activity_result = {user.id: {day: 0 for day in days} for user in users}

for user in users:
    for commit in database.query(Commit.LABEL, query=where("author_id") == user.id):
        activity_result[user.id][datetime.utcfromtimestamp(commit.timestamp).date()] += 1

## Visualisierung der Daten

Im zweiten Schritt werden die Daten anhand eines gestapelten Säulendiagramms dargestellt.

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = "svg"
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

plt.rc("font", **locals.FONT)
fig, ax = plt.subplots(figsize=(locals.TEXTWIDTH, 3.2))

dates = list(next(iter(activity_result.values())).keys())

for index, data in enumerate(activity_result.items()):
    user, user_result = data
    values = list(user_result.values())
    
    if index == 0:
        bottom = None
    else:
        bottom = np.array(list(activity_result[users[0].id].values()))

        for previous_dataset in range(1, index):
            bottom += np.array(list(activity_result[users[previous_dataset].id].values()))

    ax.bar(dates, values, bottom=bottom,
        color=locals.USERS[user]["color"], label=locals.USERS[user]["id"])

legend = ax.legend(title="Entwickler:in", loc="upper center", bbox_to_anchor=(0.5, 1.28),
    ncol=8, labelspacing=0.1, columnspacing=2, edgecolor=locals.FRAME_GRAY)
legend._legend_box.sep = 4

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=16))
ax.xaxis.set_major_formatter(mticker.NullFormatter())
ax.xaxis.set_minor_formatter(mdates.DateFormatter("%b"))
ax.tick_params(axis="x", which="minor", bottom=False, top=False, labelbottom=True)

ax.margins(x=0)
ax.set_xlabel("2020–2021")
ax.set_ylabel("Commits")
ax.yaxis.set_major_locator(mticker.MultipleLocator(base=10))
ax.grid(color=locals.GRID_GRAY)
ax.set_axisbelow(True)

plt.tight_layout()
plt.show()

## Erstellung der Wertetabelle

Als letzter Schritt wird die entsprechende Wertetabelle zum Diagramm erstellt.

In [3]:
from IPython.display import display, HTML, Markdown
from tabulate import tabulate

display(Markdown("**Wertetabelle**"))

table_headers = ["Datum"]
table_data = {date: {} for date in days}

for author, result in activity_result.items():
    table_headers.append(locals.USERS[author]["id"])
    
    for date, value in result.items():
        table_data[date][author] = value

table_rows = []

for date, result in table_data.items():
    row = [date.strftime("%d.%m.%Y")]
    row.extend(result.values())
    table_rows.append(row)

display(HTML(tabulate(headers=table_headers,
    tabular_data=table_rows,
    tablefmt="html", stralign="center", numalign="center")))

**Wertetabelle**

Datum,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
01.03.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
02.03.2020,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0
03.03.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
04.03.2020,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,0
05.03.2020,0,0,0,0,29,0,0,0,0,0,0,0,0,0,0,0
06.03.2020,0,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0
07.03.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
08.03.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
09.03.2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10.03.2020,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
